This notebook compares the performance of 4 different models using inputs from the module specification sheet and lab gathered data in the PAN file

In [1]:
%run PVModel_functions.ipynb

In [2]:
#This cell imports all the necessary packages & module data
import pandas as pd 
import numpy as np
from numpy import median
from datetime import datetime
import datatools
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import pvlib
import pvanalytics
import math
import pvpltools
import seaborn as sns
from pvpltools import iec61853
from pvpltools.module_efficiency import adr, heydenreich, motherpv, pvgis, mpm5, mpm6, bilinear
from pvpltools.module_efficiency import fit_efficiency_model, fit_bilinear

%matplotlib inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
#mpl.rcParams['font.weight']='bold'
mpl.rcParams['axes.titlesize']=22
sns.set(rc={'figure.figsize':(12,7)})

mod_names = 'LG', 'Panasonic', 'Can270', 'Can275', 'HanQPlus', 'HanQPeak', 'MissionSolar'
four_strings = ['LG', 'Panasonic', 'Can270', 'Can275', 'HanQPlus', 'HanQPeak', 'MissionSolar']
mettablename =  'dbo.SNLA_Met_DataOut'

In [3]:
#Generate empty df to store appended results
results_four=[]


In [4]:
for mod_name in mod_names:
    
        
       #Reads in an excel file of all necessary module coefficients, string lengths, system dates, etc
        mod_info = pd.read_excel('module_coefficients.xlsx')
        mod_info.index = mod_info['Manufacturer']
        del mod_info['Manufacturer']
        #defines the specific module given the previous user input and module coefficients file
        module = mod_info[mod_name]
        systablename = module['table_name']
        start = module['start']
        end = module['end']
        #year='2020'

        #Import sys & met databases & merge into one df
        df = sys_met_data(mettablename, systablename, start, end)

        #Calculate solpos & other environmental data, like airmass, aoi, and differnt poa components
        pres = df['Pressure_mBar_Avg']*100
        sdf = calc_sol_data(time=df.index, latitude=module['latitude'], longitude=module['longitude'],
                        temperature=df['Temp_C_Avg'], tilt=module['tilt'], altitude=module['altitude'])
        #Calculate Environmental Data i.e. airmass, pressure, POA irradiance compenents etc.
        edf = calc_env_data(tilt=module['tilt'],altitude=module['altitude'], sur_azimuth=module['surface_azimuth'], 
                        sol_azimuth=sdf['azimuth'], zenith=sdf['apparent_zenith'], DNI=df['Direct_Wm2_Avg'], 
                        GHI=df['Global_Wm2_Avg'], DHI=df['Diffuse_Wm2_Avg'], DNI_extra=sdf['dni_extra'], model ='haydavies')

        #Calculate airmass and add this to the Environmental Data
        edf['am_abs'] = pvlib.atmosphere.get_absolute_airmass(airmass_relative=edf['airmass'], pressure=pres)

        #Solar Elevation Filter
        sol_elev_mask = simple_filter(sdf, 'apparent_elevation', lower=15, upper=90, apply=False)
        df = df.where(sol_elev_mask, np.nan)


         #Ambient temperature filter
        df = simple_filter(sys=df, variable='Temp_C_Avg', lower=-10, upper=40, apply=True)

        #Finding Avg RTD - Systems have different strings available 
        all_rtd = ['LG', 'Panasonic', 'MissionSolar', 'Jinko']
        twth_rtd = ['HanQPlus', 'HanQPeak']
        onfo_rtd = ['Can 270', 'Can275']
        tmp = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
        
        if mod_name in all_rtd:
            df = simple_filter(sys=df, variable=module['str_1_rtd'], lower=-40, upper=85, apply=True)
            df = simple_filter(sys=df, variable=module['str_2_rtd'], lower=-40, upper=85, apply=True)
            df = simple_filter(sys=df, variable=module['str_3_rtd'], lower=-40, upper=85, apply=True)
            df = simple_filter(sys=df, variable=module['str_4_rtd'], lower=-40, upper=85, apply=True)
            
            df['mod_temp'] = ((df[module['str_1_rtd']]+df[module['str_2_rtd']]+df[module['str_3_rtd']]+df[module['str_4_rtd']])/4)
            
        elif mod_name in twth_rtd:
            df = simple_filter(sys=df, variable=module['str_2_rtd'], lower=-40, upper=85, apply=True)
            df = simple_filter(sys=df, variable=module['str_3_rtd'], lower=-40, upper=85, apply=True)
            
            df['mod_temp'] = ((df[module['str_2_rtd']]+df[module['str_3_rtd']])/2)
                        
        else:
            df = simple_filter(sys=df, variable=module['str_1_rtd'], lower=-40, upper=85, apply=True)
            df = simple_filter(sys=df, variable=module['str_4_rtd'], lower=-40, upper=85, apply=True)
                        
            df['mod_temp'] = ((df[module['str_1_rtd']]+df[module['str_4_rtd']])/2)
            
                              
        df['cell_temp'] = pvlib.temperature.sapm_cell_from_module(module_temperature=df['mod_temp'], poa_global=edf['poa_global'], deltaT = tmp['deltaT'] )                  
        df = simple_filter(sys=df,variable='cell_temp', lower=-40, upper=85, apply=True)
        
        df['eff_irr'] = pvlib.pvsystem.sapm_effective_irradiance(poa_direct=edf['poa_direct'], poa_diffuse=edf['poa_diffuse'], airmass_absolute=edf['am_abs'], aoi=edf['aoi'], module=module)      
        df = simple_filter(sys=df, variable='eff_irr', lower=50, upper=1200, apply=True)
                           
        
        #Removing individual days that cause large errors due to shading, snow, etc

        #filter out pole shading 
        shade_list = ['Can270', 'HanQPlus']
        if mod_name in shade_list:
            shade_start = '2020-10-06 00:00:00-07:00'
            shade_end = '2020-11-02 00:00:00-07:00'
            shask = (df.index > shade_start) & (df.index <= shade_end)
            df = df.loc[~shask]

        #Filtering out days in 2019 that caused large outliers - could be due to snow on modules    
        dec_list = ['Panasonic', 'LG']
        if mod_name in dec_list:
            dec_start = '2019-12-29 00:00:00-07:00'
            dec_end = '2019-12-30 00:0:00-07:00'
            dec_mask = (df.index > dec_start) & (df.index <= dec_end)
            df = df.loc[~dec_mask]

        #remove feb 23 2019
        feb_start = '2019-02-23 00:00:00-07:00'
        feb_end = '2019-02-24 10:00:00-07:00'
        feb_mask = (df.index > feb_start) & (df.index <= feb_end)
        df = df.loc[~feb_mask] 

        #remove dec 2 2018
        dec_start = '2018-12-02 00:00:00-07:00'
        dec_end = '2018-12-02 23:59:00-07:00'
        dec_mask = (df.index > dec_start) & (df.index <= dec_end)
        df = df.loc[~dec_mask]

        #remove dec 26 2018
        dec_start = '2018-12-26 00:00:00-07:00'
        dec_end = '2018-12-26 23:59:00-07:00'
        dec_mask = (df.index > dec_start) & (df.index <= dec_end)
        df = df.loc[~dec_mask]

        #Removing Days in 2020 that had extremely large differences 
        #remove sep 21 2020
        sep_start = '2020-09-21 00:00:00-07:00'
        sep_end = '2020-09-22 23:00:00-07:00'
        sep_mask = (df.index > sep_start) & (df.index <= sep_end)
        df = df.loc[~sep_mask]

        #remove feb 5 2020
        feb_start = '2020-02-05 00:00:00-07:00'
        feb_end = '2020-02-05 23:00:00-07:00'
        feb_mask = (df.index > feb_start) & (df.index <= feb_end)
        df = df.loc[~feb_mask]


        ##### Filter out the data points with a certain ratio of current to irradiance
        string_is ='str_1_i', 'str_2_i', 'str_3_i', 'str_4_i'
        ratio_names = 'ratio str_1_i to eff_irr', 'ratio str_2_i to eff_irr', 'ratio str_3_i to eff_irr', 'ratio str_4_i to eff_irr'

        for string_i in string_is:
            #Calculate the ratio at each given timestep
            df['ratio' + ' ' + string_i + ' ' 'to eff_irr'] = df[module[string_i]]/df['eff_irr']
            #Create a df with no NaNs to calculate the slope of trendline between current and irradiance
            nadf = df.dropna(inplace=False)
            z = np.polyfit(nadf['eff_irr'], nadf[module[string_i]],1)
            #Calculate lower/upper bounds
            lower = z[0]-(z[0]*0.3)
            upper = (z[0]*0.3) + z[0]
            #Use previously defined column name to filter values
            ratio_name ='ratio' + ' ' + string_i + ' ' 'to eff_irr'
            df = simple_filter(df, ratio_name, lower=lower, upper=upper, apply=True)  

        ##There are 3 iterations of each model being run. First is the results with the adjusted PAN File, 
        ##second are the results with the original CFV PAN File Third are the results with the module specification sheet data

       #Generate a results dataframe and add measured values
        str_1 = meas_val(df[module['str_1_v']],df[module['str_1_i']])
        str_2 = meas_val(df[module['str_2_v']],df[module['str_2_i']])
        str_3 = meas_val(df[module['str_3_v']],df[module['str_3_i']])
        str_4 = meas_val(df[module['str_4_v']],df[module['str_4_i']])
        str_5 = np.nan

        #PVWATTS ADJUSTED PAN FILE
        pvw_results = ((pvwatts(POA_eff = df['eff_irr'], cell_temp = df['cell_temp'], stc_mod_p = module['Power'], 
            Gpmp = (module['gamma_pmp']/100),str_len = module['str_len'],temp_ref=25.0 ))/module['pmp_gain']).to_frame()
        

        pvw_results.rename(columns = {0 : 'Modeled Power'}, inplace = True)
        pvw_results['Str_1'] = str_1
        pvw_results['Str_2'] = str_2
        pvw_results['Str_3'] = str_3
        pvw_results['Str_4'] = str_4
        pvw_results['Str_5'] = str_5
        pvw_results['Time'] = pvw_results.index
        pvw_results['Year'] = pvw_results.index.year
        pvw_results['Model Name'] = 'PVW'
        pvw_results['System'] = mod_name
        pvw_results['Source'] = 'Case 1'
        results_four.append(pvw_results)

       #PVWATTS ORIGINAL PAN FILE
        apvw_results = ((pvwatts(POA_eff = df['eff_irr'], cell_temp = df['cell_temp'], stc_mod_p = module['Power'], 
            Gpmp = (module['gamma_pmp']/100),str_len = module['str_len'],temp_ref=25.0 ))).to_frame()
        

        apvw_results.rename(columns = {0 : 'Modeled Power'}, inplace = True)
        apvw_results['Str_1'] = str_1
        apvw_results['Str_2'] = str_2
        apvw_results['Str_3'] = str_3
        apvw_results['Str_4'] = str_4
        apvw_results['Str_5'] = str_5
        apvw_results['Time'] = apvw_results.index
        apvw_results['Year'] = apvw_results.index.year
        apvw_results['Model Name'] = 'PVW'
        apvw_results['System'] = mod_name
        apvw_results['Source'] = 'Case 2'
        results_four.append(apvw_results)
        
    
        
        #PVWATTS SPECIFICATION SHEET
        spvw_results = (pvwatts(POA_eff = df['eff_irr'], cell_temp = df['cell_temp'], stc_mod_p = module['Power'], 
        Gpmp = (module['Gampmp']/100),str_len = module['str_len'],temp_ref=25.0 )).to_frame()

        spvw_results.rename(columns = {0 : 'Modeled Power'}, inplace = True)
        spvw_results['Str_1'] = str_1
        spvw_results['Str_2'] = str_2
        spvw_results['Str_3'] = str_3
        spvw_results['Str_4'] = str_4
        spvw_results['Str_5'] = str_5
        spvw_results['Time'] = spvw_results.index
        spvw_results['Year'] = spvw_results.index.year
        spvw_results['Model Name'] = 'PVW'
        spvw_results['System'] = mod_name
        spvw_results['Source'] = 'Case 3'
        results_four.append(spvw_results)

        
        
        #RETRIEVING CEC COEFFICIENTS USING FIT CEC SAM
        #GETTING COEFFICIENTS USING PAN FILE VALUES
        cec_ivt = pvlib.ivtools.sdm.fit_cec_sam(celltype=module['cell_type'], v_mp=module['v_mp'], i_mp=module['i_mp'],
                                    v_oc=module['v_oc'], i_sc=module['i_sc'],alpha_sc=module['alpha_sc'], 
                                    beta_voc=module['beta_voc'],gamma_pmp=(module['gamma_pmp']),
                                    cells_in_series=module['Cells_in_Series'])
        
        #GETTING COEFFICIENTS USING SPEC SHEET VALUES
        cec_ivts = pvlib.ivtools.sdm.fit_cec_sam(celltype=module['cell_type'], v_mp=module['Vmpp'], i_mp=module['Impp'],
                                    v_oc=module['Voc'], i_sc=module['Isc'],alpha_sc=module['Alisc'], 
                                    beta_voc=module['Bvoc'],gamma_pmp=(module['Gampmp']),
                                    cells_in_series=module['Cells_in_Series'])
        
        #cec_ivt is IL, Io, Rs, Rshref, a_ref, and Adjust

        
        #CEC USING ADJUSTED PAN FILE VALUES
        cec_param = pvlib.pvsystem.calcparams_cec(effective_irradiance=df['eff_irr'], 
                temp_cell=df['cell_temp'], alpha_sc=module['alpha_sc'],a_ref=cec_ivt[4], I_L_ref=cec_ivt[0], 
                I_o_ref=cec_ivt[1], R_sh_ref=cec_ivt[3],R_s=cec_ivt[2],Adjust=cec_ivt[5],EgRef=1.121)
        #Calculate power using single diode model
        cec_p = pvlib.pvsystem.singlediode(photocurrent = cec_param[0], saturation_current = cec_param[1], resistance_series =
                               cec_param[2], resistance_shunt = cec_param[3], nNsVth = cec_param[4], method = 'newton')
        #Calculate string power
        cec_results = (sdm(p_mp=cec_p['p_mp'], str_len=module['str_len'])/module['pmp_gain']).to_frame()


        cec_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        cec_results['Str_1'] = str_1
        cec_results['Str_2'] = str_2
        cec_results['Str_3'] = str_3
        cec_results['Str_4'] = str_4
        cec_results['Str_5'] = str_5
        cec_results['Time'] = cec_results.index
        cec_results['Year'] = cec_results.index.year
        cec_results['Model Name'] = 'CEC'
        cec_results['System'] = mod_name
        cec_results['Source'] = 'Case 1'
        results_four.append(cec_results)
        
        
        #CEC USING ORIGINAL CFV PAN FILE VALUES
        acec_results = (sdm(p_mp=cec_p['p_mp'], str_len=module['str_len'])).to_frame()


        acec_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        acec_results['Str_1'] = str_1
        acec_results['Str_2'] = str_2
        acec_results['Str_3'] = str_3
        acec_results['Str_4'] = str_4
        acec_results['Str_5'] = str_5
        acec_results['Time'] = acec_results.index
        acec_results['Year'] = acec_results.index.year
        acec_results['Model Name'] = 'CEC'
        acec_results['System'] = mod_name
        acec_results['Source'] = 'Case 2'
        results_four.append(acec_results)
        
        
        #CEC USING MODULE SPECIFICATION SHEET VALUES
        cec_param = pvlib.pvsystem.calcparams_cec(effective_irradiance=df['eff_irr'], 
                temp_cell=df['cell_temp'], alpha_sc=module['Alisc'],a_ref=cec_ivts[4], I_L_ref=cec_ivts[0], 
                I_o_ref=cec_ivts[1], R_sh_ref=cec_ivts[3],R_s=cec_ivts[2],Adjust=cec_ivts[5],EgRef=1.121)
        #Calculate power using single diode model
        cec_p = pvlib.pvsystem.singlediode(photocurrent = cec_param[0], saturation_current = cec_param[1], resistance_series =
                               cec_param[2], resistance_shunt = cec_param[3], nNsVth = cec_param[4], method = 'newton')
        #Calculate string power
        scec_results = sdm(p_mp=cec_p['p_mp'], str_len=module['str_len']).to_frame()


        scec_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        scec_results['Str_1'] = str_1
        scec_results['Str_2'] = str_2
        scec_results['Str_3'] = str_3
        scec_results['Str_4'] = str_4
        scec_results['Str_5'] = str_5
        scec_results['Time'] = scec_results.index
        scec_results['Year'] = scec_results.index.year
        scec_results['Model Name'] = 'CEC'
        scec_results['System'] = mod_name
        scec_results['Source'] = 'Case 3'
        results_four.append(scec_results)

        
        
        #DESOTO / PVSYST PARAMETER CALCULATION USING PAN FILE VALUES
        #Calculate Io
        I_o = ((module['i_sc']+((module['i_sc']*module['R_s'])/module['R_sh_ref']))-(module['v_oc']/module['R_sh_ref']))/\
        (np.exp((module['v_oc']*(1.60217662e-19))/(module['gamma_ref']*module['Cells_in_Series']*(1.38064852e-23)*(298.15)))-1)
        #Calculate I_L
        I_L = module['i_sc'] + ((module['i_sc']*module['R_s'])/module['R_sh_ref'])
        a_ref = ((module['gamma_ref']*module['Cells_in_Series']*(((1.38064852e-23)*(298.15))/(1.60217662e-19))))


        ds_param = pvlib.pvsystem.calcparams_desoto(effective_irradiance=df['eff_irr'], 
                temp_cell=df['cell_temp'], alpha_sc=module['alpha_sc'],a_ref=a_ref, I_L_ref=I_L, 
                I_o_ref=I_o, R_sh_ref=module['R_sh_ref'],R_s=module['R_s'], EgRef=1.121)
        
        #DESOTO USING ADJUSTED PAN FILE
        ds_p = pvlib.pvsystem.singlediode(photocurrent = ds_param[0], saturation_current = ds_param[1], resistance_series =
                               ds_param[2], resistance_shunt = ds_param[3], nNsVth = ds_param[4], method = 'newton')
        #Calculate string power
        des_results = (sdm(p_mp=ds_p['p_mp'], str_len=module['str_len'])/module['pmp_gain']).to_frame()

        des_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        des_results['Str_1'] = str_1
        des_results['Str_2'] = str_2
        des_results['Str_3'] = str_3
        des_results['Str_4'] = str_4
        des_results['Str_5'] = str_5
        des_results['Time'] = des_results.index
        des_results['Year'] = des_results.index.year
        des_results['Model Name'] = 'DES'
        des_results['System'] = mod_name
        des_results['Source'] = 'Case 1'
        results_four.append(des_results)
        
        
        #DESOTO USING ORIGINIAL CFV PAN FILE
        ades_results = (sdm(p_mp=ds_p['p_mp'], str_len=module['str_len'])).to_frame()

        ades_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        ades_results['Str_1'] = str_1
        ades_results['Str_2'] = str_2
        ades_results['Str_3'] = str_3
        ades_results['Str_4'] = str_4
        ades_results['Str_5'] = str_5
        ades_results['Time'] = ades_results.index
        ades_results['Year'] = ades_results.index.year
        ades_results['Model Name'] = 'DES'
        ades_results['System'] = mod_name
        ades_results['Source'] = 'Case 2'
        results_four.append(ades_results)

        
        #DESOTO USING MODULE SPECIFICATION SHEET
        ds_param = pvlib.pvsystem.calcparams_desoto(effective_irradiance=df['eff_irr'], 
                temp_cell=df['cell_temp'], alpha_sc=module['Alisc'],a_ref=cec_ivts[4], I_L_ref=cec_ivts[0], 
                I_o_ref=cec_ivts[1], R_sh_ref=cec_ivts[3],R_s=cec_ivts[2], EgRef=1.121)
        #Calculate power using single diode model
        ds_p = pvlib.pvsystem.singlediode(photocurrent = ds_param[0], saturation_current = ds_param[1], resistance_series =
                               ds_param[2], resistance_shunt = ds_param[3], nNsVth = ds_param[4], method = 'newton')
        #Calculate string power
        sdes_results = sdm(p_mp=ds_p['p_mp'], str_len=module['str_len']).to_frame()

        sdes_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        sdes_results['Str_1'] = str_1
        sdes_results['Str_2'] = str_2
        sdes_results['Str_3'] = str_3
        sdes_results['Str_4'] = str_4
        sdes_results['Str_5'] = str_5
        sdes_results['Time'] = sdes_results.index
        sdes_results['Year'] = sdes_results.index.year
        sdes_results['Model Name'] = 'DES'
        sdes_results['System'] = mod_name
        sdes_results['Source'] = 'Case 3'
        results_four.append(sdes_results)
       #Calculte PVSystem results and add to df

    
       
    
      
        
        #PVSYST USING ADJUSTED PAN FILE
        #Calculate parameters necessary to input into single diode model
        pvs_param = pvlib.pvsystem.calcparams_pvsyst(effective_irradiance=df['eff_irr'],temp_cell=df['cell_temp'], 
                alpha_sc=module['alpha_sc'],gamma_ref=module['gamma_ref'],mu_gamma=module['mu_gamma'], 
                I_L_ref=I_L,I_o_ref=I_o,R_sh_ref=module['R_sh_ref'], R_sh_0=module['R_sh_0'],
                R_s=module['R_s'],cells_in_series=module['Cells_in_Series'], EgRef=1.121)
        #Calculate power using single diode model
        pvs_p = pvlib.pvsystem.singlediode(photocurrent = pvs_param[0], saturation_current = pvs_param[1], resistance_series =
                               pvs_param[2], resistance_shunt = pvs_param[3], nNsVth = pvs_param[4], method = 'newton')
        #Calculate string power
        pvs_results = (sdm(p_mp=pvs_p['p_mp'], str_len=module['str_len'])/module['pmp_gain']).to_frame()

        pvs_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        pvs_results['Str_1'] = str_1
        pvs_results['Str_2'] = str_2
        pvs_results['Str_3'] = str_3
        pvs_results['Str_4'] = str_4
        pvs_results['Str_5'] = str_5
        pvs_results['Time'] = pvs_results.index
        pvs_results['Year'] = pvs_results.index.year
        pvs_results['Model Name'] = 'PVS'
        pvs_results['System'] = mod_name
        pvs_results['Source'] = 'Case 1'
        results_four.append(pvs_results)
        
        #PVSYST USING ORIGINAL CFV PAN FILE
        apvs_results = (sdm(p_mp=pvs_p['p_mp'], str_len=module['str_len'])).to_frame()

        apvs_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        apvs_results['Str_1'] = str_1
        apvs_results['Str_2'] = str_2
        apvs_results['Str_3'] = str_3
        apvs_results['Str_4'] = str_4
        apvs_results['Str_5'] = str_5
        apvs_results['Time'] = apvs_results.index
        apvs_results['Year'] = apvs_results.index.year
        apvs_results['Model Name'] = 'PVS'
        apvs_results['System'] = mod_name
        apvs_results['Source'] = 'Case 2'
        results_four.append(apvs_results)

        
        #PVSYST USING MODULE SPEC SHEET 
        a_ref=cec_ivts[4]
        gamma_ref = (a_ref)/((module['Cells_in_Series'])*(((1.38064852e-23)*(298.15))/(1.60217662e-19)))
        mu_gamma = -0.0003
        
        #Calculate parameters necessary to input into single diode model
        pvs_param = pvlib.pvsystem.calcparams_pvsyst(effective_irradiance=df['eff_irr'],temp_cell=df['cell_temp'], 
                alpha_sc=module['Alisc'],gamma_ref=gamma_ref,mu_gamma=mu_gamma, 
                I_L_ref=cec_ivts[0],I_o_ref=cec_ivts[1],R_sh_ref=cec_ivts[3], R_sh_0=(10*cec_ivts[3]),
                R_s=cec_ivts[2],cells_in_series=module['Cells_in_Series'], EgRef=1.121)
        #Calculate power using single diode model
        pvs_p = pvlib.pvsystem.singlediode(photocurrent = pvs_param[0], saturation_current = pvs_param[1], resistance_series =
                               pvs_param[2], resistance_shunt = pvs_param[3], nNsVth = pvs_param[4], method = 'newton')
        #Calculate string power
        spvs_results = sdm(p_mp=pvs_p['p_mp'], str_len=module['str_len']).to_frame()

        spvs_results.rename(columns = {'p_mp' : 'Modeled Power'}, inplace = True)
        spvs_results['Str_1'] = str_1
        spvs_results['Str_2'] = str_2
        spvs_results['Str_3'] = str_3
        spvs_results['Str_4'] = str_4
        spvs_results['Str_5'] = str_5
        spvs_results['Time'] = spvs_results.index
        spvs_results['Year'] = spvs_results.index.year
        spvs_results['Model Name'] = 'PVS'
        spvs_results['System'] = mod_name
        spvs_results['Source'] = 'Case 3'
        results_four.append(spvs_results)

        del df
        del pvw_results
        del cec_results
        del des_results
        del pvs_results
    

        print(mod_name)
        
        
    

C:\Users\lmdevil\Anaconda3\envs\PVModel\lib\site-packages\pvlib\solarposition.py:368: FutureWarning: casting datetime64[ns, MST] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  unixtime = np.array(time.astype(np.int64)/10**9)


LG
Panasonic
Can270
Can275
HanQPlus
HanQPeak
MissionSolar


In [5]:
#Gathers the results from every system with four strings
meta4sp = pd.concat(results_four, ignore_index=True)
meta4sp.index=meta4sp['Time']

In [6]:
meta4sp.to_hdf('meta4sp.h5', key='data', complib='zlib', complevel=5)